In [1]:
using PCT

[ Info: Precompiling PCT [ef708a43-c8a3-43f4-8f65-1f04ee4c5bb0]


## $x^{\dagger} A x$

In [66]:
f, _ = @pct (A::Her) -> pullback((x::CV) -> sum((i, j), x(i)' * A(i, j) * x(j))); f

(A) -> 
    𝒫((x) -> 
        ∑((i, j), (x(i)'⋅x(j)⋅A(i, j))))

In [67]:
df = vdiff(f; settings=symmetry_settings)

(A) -> 
    (x) -> 
        (_d) -> 
            ∑((j), (x(j)⋅A(_d, j)⋅2.0))

## Trace of Product

In [141]:
f, _ = @pct let tr = (X::CM) -> sum(i, X(i, i)),
    matmul = (A::CM, B::CM) -> (i::I, j::I) -> sum(n, A(i, n) * B(n, j))
    pullback((A::CM, B::CM) -> tr(matmul(A, B) + matmul(B, A)))
end; f

let tr = (X) -> 
    ∑((i), X(i, i))
matmul = (A, B) -> 
    (i, j) -> 
        ∑((n), (A(i, n)⋅B(n, j)))
𝒫((A, B) -> 
    tr((matmul(A, B)+matmul(B, A))))
end

In [142]:
vdiff(eval_all(f); settings=symmetry_settings)

(A, B) -> 
    (_d, _d_1) -> 
        (B(_d_1, _d)'⋅2.0), (_d, _d_1) -> 
        (A(_d_1, _d)'⋅2.0)

## Conjugate Gradient

In [143]:
f, _ = @pct (A::Sym, r::RV, p::RV, b::RV, x::RV) -> let E = (x::RV) -> sum((i, j), 0.5 * x(i) * A(i, j) * x(j)) + sum(i, x(i) * b(i))
    pullback((α::R, β::R) -> E((i::I) -> x(i) + α * (r(i) + β * p(i))))
end; f

(A, r, p, b, x) -> 
    let E = (x) -> 
        (∑((i), (x(i)⋅b(i)))+∑((i, j), (x(i)⋅x(j)⋅A(i, j)⋅0.5)))
    𝒫((α, β) -> 
        E((i) -> 
            ((((p(i)⋅β)+r(i))⋅α)+x(i))))
    end

In [146]:
df = vdiff(f)

(A, r, p, b, x) -> 
    let E = (x) -> 
        (∑((i), (x(i)⋅b(i)))+∑((i, j), (x(i)⋅x(j)⋅A(i, j)⋅0.5)))
    (α, β) -> 
        ∑((i), (((p(i)⋅β)+r(i))⋅𝒫(E)((i) -> 
            ((((p(i)⋅β)+r(i))⋅α)+x(i)), 1)(i))), ∑((i), (p(i)⋅𝒫(E)((i) -> 
            ((((p(i)⋅β)+r(i))⋅α)+x(i)), 1)(i)⋅α))
    end

In [147]:
df = vdiff(eval_all(f); settings=symmetry_settings)

(A, r, p, b, x) -> 
    (α, β) -> 
        (∑((i), (((p(i)⋅β)+r(i))⋅b(i)))+∑((i, j), (((((p(j)⋅β)+r(j))⋅α)+x(j))⋅((p(i)⋅β)+r(i))⋅A(i, j)))), (∑((i), (b(i)⋅p(i)⋅α))+∑((i, j), (((((p(j)⋅β)+r(j))⋅α)+x(j))⋅p(i)⋅A(i, j)⋅α)))

## Lagrangian Mechanics

In [68]:
_, ctx = @pct begin
    @space X begin
        type = (RF, ) -> RF
        linear = true
    end
end; 

In [69]:
f = @pct _ ctx (L::RO, D::X) -> pullback((r::RF) -> sum((t::R), L(r(t), D(r)(t)))); f

(L, D) -> 
    𝒫((r) -> 
        ∑((t), L(r(t), D(r)(t))))

In [70]:
vdiff(f; settings=custom_settings(:clench_sum=>true))

(L, D) -> 
    (r) -> 
        (D'((_d) -> 
            𝒫((_z_1) -> 
                L(r(_d), _z_1))(D(r)(_d), 1))+(_d) -> 
            𝒫((_z_2) -> 
                L(_z_2, D(r)(_d)))(r(_d), 1))

## Hartree Fock Gradient and Hessian Vector Product

In [71]:
_, ctx = @pct begin    
    @space T begin
        type = (I, I, I, I) -> C
        symmetries = (((2, 1, 4, 3), :conj), ((3, 4, 1, 2), :id))
    end
end;

In [72]:
f = @pct _ ctx (J::T) -> pullback((C::CM) -> sum((i, j, p, q, r, s),
    C(p, i)' * C(q, i) * C(r, j)' * C(s, j) * J(p, q, r, s)))

(J) -> 
    𝒫((C) -> 
        ∑((i, j, p, q, r, s), (C(p, i)'⋅C(r, j)'⋅J(p, q, r, s)⋅C(s, j)⋅C(q, i))))

In [73]:
df = vdiff(f; settings=symmetry_settings)

(J) -> 
    (C) -> 
        (_d, _d_1) -> 
            ∑((i, p, q, r), (C(q, i)'⋅J(_d, r, q, p)⋅C(p, i)⋅C(r, _d_1)⋅4.0))

In [74]:
df |> fc |> decompose |> pp |> simplify |> first

(C, _k) -> 
    (_a_2, _a_3) -> 
        (∑((_d, i, p, q), (J(_d, _a_2, q, p)'⋅C(p, i)'⋅_k(_d, _a_3)⋅C(q, i)⋅4.0))+∑((_d, _d_1, q, r), (J(_d, r, q, _a_2)'⋅C(r, _d_1)'⋅_k(_d, _d_1)⋅C(q, _a_3)⋅4.0))+∑((_d, _d_1, p, r), (_k(_d, _d_1)'⋅J(_d, r, _a_2, p)⋅C(p, _a_3)⋅C(r, _d_1)⋅4.0)))

## Foster Boys

In [157]:
f, _ = @pct (R::Her, R2::Her) -> pullback((C::CM) -> 
let r = (i::I) -> sum((p, q), R(p, q) * C(i, p)' * C(i, q))
    sum(i, sum((p, q), R2(p, q) * C(i, p)' * C(i, q)) - r(i)' * r(i))
    end); f

(R, R2) -> 
    𝒫((C) -> 
        let r = (i) -> 
            ∑((p, q), (C(i, p)'⋅R(p, q)⋅C(i, q)))
        ∑((i), ((r(i)'⋅r(i)⋅-1.0)+∑((p, q), (C(i, p)'⋅R2(p, q)⋅C(i, q)))))
        end)

In [162]:
df = vdiff(eval_all(f); settings=symmetry_settings)

(R, R2) -> 
    (C) -> 
        (_d, _d_1) -> 
            (∑((p), (R2(_d_1, p)⋅C(_d, p)⋅2.0))+∑((p, _p, q), (C(_d, _p)'⋅R(_d_1, p)⋅R(_p, q)⋅C(_d, q)⋅C(_d, p)⋅-4.0)))

## Localized Wannier Functions

In [89]:
_, ctx = @pct begin
    @domain P begin
        base = R
        periodic = true
    end
    
    @domain Q begin
        contractable = false
    end
    
    @space Mmn begin
        type = (I, I, R, R) -> C
        symmetries = (((2, 1, 4, 3), :conj),)
    end

    @space SV begin
        type = (I,) -> C
        symmetries = (((1,), :ineg),)
    end
    
    @space Gauge begin
        type = (I, I, R) -> C
    end
end;

In [90]:
f = @pct _ ctx (S::Mmn, w::SV) -> pullback((U::Gauge) -> 
    let ρ = (n::I, b::Q) -> sum((k::P, p, q), U(p, n, k)' * S(p, q, k, k + b) * U(q, n, k + b))
        sum((n, b::Q), ρ(n, b)' * ρ(n, b))
    end)

(S, w) -> 
    𝒫((U) -> 
        let ρ = (n, b) -> 
            ∑((k, p, q), (U(p, n, k)'⋅U(q, n, (b+k))⋅S(p, q, k, (b+k))))
        ∑((n, b), (ρ(n, b)'⋅ρ(n, b)))
        end)

In [91]:
df = vdiff(eval_all(f); settings=symmetry_settings)

(S, w) -> 
    (U) -> 
        (_d, _d_1, _d_2) -> 
            ∑((b, p, k, _p, q), (U(_p, _d_1, (b+k))'⋅U(q, _d_1, k)⋅U(p, _d_1, (_d_2+b))⋅S(_d, p, _d_2, (_d_2+b))⋅S(_p, q, (b+k), k)⋅4.0))